In [ ]:
import pickle as pkl
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd

In [ ]:
BASELINE, BAYES = 0, 1
BONCORRECT_COEF = 2
names = [["ImageNet"]*40, ["MNIST"]*40, ["German"]*40, ["Compas"]*40]

In [ ]:
lime_stability = []
with open("imagenet_blime_stability", "rb") as f:
    lime_stability.append(pkl.load(f))
with open("mnist_4_blime_stability", "rb") as f:
    lime_stability.append(pkl.load(f))
with open("german_blime_stability", "rb") as f:
    lime_stability.append(pkl.load(f))
with open("compas_blime_stability", "rb") as f:
    lime_stability.append(pkl.load(f))
    
shap_stability = []
with open("imagenet_bshap_stability", "rb") as f:
    shap_stability.append(pkl.load(f))
with open("mnist_4_bshap_stability", "rb") as f:
    shap_stability.append(pkl.load(f))
with open("german_bshap_stability", "rb") as f:
    shap_stability.append(pkl.load(f))
with open("german_blime_stability", "rb") as f:
    shap_stability.append(pkl.load(f))
lime_stability = np.array(lime_stability)
shap_stability = np.array(shap_stability)

In [ ]:
def get_pct_change(baseline, bayes):
    baseline, bayes = baseline.tolist(), bayes.tolist()
    pcts = []
    for bs, by in zip(baseline, bayes):
        dif = bs - by
        abs_dif = abs(dif)
        if dif > 0:
            pcts.append(abs_dif / bs)
        else:
            pcts.append(-1 * abs_dif / bs)
    return np.array(pcts) * 100

In [ ]:
# LIME 
lime_pct_changes = []
for i in range(lime_stability.shape[0]):
    lime_pct_changes.append(get_pct_change(lime_stability[i][BASELINE], lime_stability[i][BAYES]))
    print("Wilcoxon Alternative Less:", BONCORRECT_COEF * stats.wilcoxon(lime_stability[i][BASELINE], lime_stability[i][BAYES], alternative='greater')[1])
    print("Wilcoxon Alternative Equal:", BONCORRECT_COEF * stats.wilcoxon(lime_stability[i][BASELINE], lime_stability[i][BAYES])[1])

In [ ]:
# SHAP
shap_pct_changes = []
for i in range(shap_stability.shape[0]):
    shap_pct_changes.append(get_pct_change(shap_stability[i][BASELINE], shap_stability[i][BAYES]))
    print("Wilcoxon Alternative Less:", BONCORRECT_COEF * stats.wilcoxon(shap_stability[i][BASELINE], shap_stability[i][BAYES], alternative='greater')[1])
    print("Wilcoxon Alternative Equal:", BONCORRECT_COEF * stats.wilcoxon(shap_stability[i][BASELINE], shap_stability[i][BAYES])[1])

In [ ]:
lime_df = pd.DataFrame()
lime_df['names'] = np.concatenate(names)
lime_df['BayesLIME % Increase in Stability'] = np.concatenate(lime_pct_changes)

plt.cla()
sns.set(font_scale=2.2, style="whitegrid")
plt.figure(figsize=(10,4))

ax = sns.boxplot(x='BayesLIME % Increase in Stability', y='names', data=lime_df, color='skyblue', fliersize=0)
ax = sns.stripplot(x='BayesLIME % Increase in Stability', y='names', data=lime_df, color='b', size=5, alpha=0.8)

plt.plot([0,0],[-5,5], color='r', linestyle="--", linewidth=3.0)
ax.set(ylabel=None)
ax.grid(False)
ax.set(ylim=[-1,4])
plt.tight_layout()
plt.savefig(f'stability_lime.pdf')

In [ ]:
shap_df = pd.DataFrame()
shap_df['names'] = np.concatenate(names)
shap_df['BayesSHAP % Increase in Stability'] = np.concatenate(shap_pct_changes)

plt.cla()
sns.set(font_scale=2.2, style="whitegrid")
plt.figure(figsize=(10,4))

ax = sns.boxplot(x='BayesSHAP % Increase in Stability', y='names', data=shap_df, color='skyblue', fliersize=0)
ax = sns.stripplot(x='BayesSHAP % Increase in Stability', y='names', data=shap_df, color='b', size=5, alpha=0.8)

plt.plot([0,0],[-5,5], color='r', linestyle="--", linewidth=3.0)
ax.set(ylabel=None)
ax.grid(False)
ax.set(ylim=[-1,4])
plt.tight_layout()
plt.savefig(f'stability_shap.pdf')